<a href="https://colab.research.google.com/github/mgrzegorz/first/blob/master/kaggle_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting Google Drive

In [228]:
!pip install sklearn

  Could not find a version that satisfies the requirement sklearn==0.20.1 (from versions: 0.0)
No matching distribution found for sklearn==0.20.1


In [129]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
# zmiana working directory
os.chdir("/content/gdrive/My Drive/Data/Elo/")

In [0]:
#source: https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
#XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [184]:
#%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c elo-merchant-category-recommendation -p /content/gdrive/My\ Drive/Data

/bin/bash: kaggle: command not found


In [0]:
merchants_types = {
  'merchant_group_id': 'uint32',
  'merchant_category_id':'int16',
  'city_id': 'int16',
  'state_id': 'int8',
  'subsector_id': 'int8',
  'numerical_1': 'float16',
  'numerical_2': 'float16',
  'category_1': 'category', #bool_,
  'category_2': 'category',
  'category_4': 'category',   
  'most_recent_sales_range': 'category',
  'most_recent_purchases_range': 'category',
  'avg_sales_lag3': 'float32',
  'avg_purchases_lag3': 'float32',
  'active_months_lag3': 'float16',
  'avg_sales_lag6': 'float32',
  'avg_purchases_lag6': 'float32',
  'active_months_lag6': 'float16',
  'avg_sales_lag12': 'float32',
  'avg_purchases_lag12': 'float32',
  'active_months_lag12': 'float16'
 }
types_new_merchant_transactions = {
  'merchant_id':'object',
  'authorized_flag':'category',
  'city_id': 'int16',
  'state_id': 'int8',    
  'subsector_id': 'int8',
  'category_1': 'category',
  'category_2': 'category',
  'category_3': 'category',
  'installments': 'int16',
  'merchant_category_id': 'int16',
  'month_lag': 'int8', #1,2 
  'purchase_amount': 'float32',
  'purchase_date': 'object'}
types_historical_transactions = {
  'authorized_flag':'category',
  'city_id': 'int16',
  'state_id': 'int8',
  'subsector_id': 'int8',
  'category_1': 'category',
  'category_2': 'category',
  'category_3': 'category',    
  'installments': 'int16',
  'merchant_category_id': 'int16',     
  'merchant_id':'object',
  'month_lag': 'int8',   
  'purchase_amount': 'float32',
  'purchase_date': 'object'
}
types_train ={
    'first_active_month':'object',
    'feature_1': 'uint8',
    'feature_2': 'uint8',
    'feature_3': 'bool_',
    'target': 'float16'
}
types_test ={
    'first_active_month':'object',
    'feature_1': 'uint8',
    'feature_2': 'uint8',
    'feature_3': 'bool_'
}

In [0]:
#new_merchant_transactions = pd.read_csv("new_merchant_transactions.csv", index_col = 'car_id', dtype=types_new_merchant_transactions)
#new_merchant_transactions['purchase_date'] = pd.to_datetime(new_merchant_transactions['purchase_date'], unit='ns')
#historical_transactions = pd.read_csv("historical_transactions.csv", index_col = 'card_id', dtype=types_historical_transactions)
#historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'], unit='ns')
#train = pd.read_csv("train.csv", index_col = 'card_id', dtype = types_train)
#train['first_active_month'] = pd.to_datetime(train['first_active_month'])
#test = pd.read_csv("test.csv", index_col = 'card_id', dtype = types_test)
#test['first_active_month'] = pd.to_datetime(test['first_active_month'])

In [0]:
merchants = pd.read_csv("merchants.csv", index_col='merchant_id', dtype = merchants_types)
merchants = pd.get_dummies(merchants, columns = ['category_1', 'category_2', 'category_4', 'state_id'])
merchants['most_recent_purchases_range'] = merchants['most_recent_purchases_range'].cat.codes #ordinal
merchants['most_recent_sales_range'] = merchants['most_recent_sales_range'].cat.codes         #ordinal

In [0]:
df = pd.merge(train, historical_transactions, on='card_id', how='left')

In [261]:
merchants.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334696 entries, M_ID_838061e48c to M_ID_cd2c0b07e9
Data columns (total 51 columns):
merchant_group_id              334696 non-null uint32
merchant_category_id           334696 non-null int16
subsector_id                   334696 non-null int8
numerical_1                    334696 non-null float16
numerical_2                    334696 non-null float16
most_recent_sales_range        334696 non-null int8
most_recent_purchases_range    334696 non-null int8
avg_sales_lag3                 334683 non-null float32
avg_purchases_lag3             334696 non-null float32
active_months_lag3             334696 non-null float16
avg_sales_lag6                 334683 non-null float32
avg_purchases_lag6             334696 non-null float32
active_months_lag6             334696 non-null float16
avg_sales_lag12                334683 non-null float32
avg_purchases_lag12            334696 non-null float32
active_months_lag12            334696 non-null float16
cit

In [0]:
train.shape